# Mesh generation for tutorial 06, case 1b

This file generates the mesh which is used in the following examples:
* 1b_poisson

The test case is from section 5.1 of
```
F. Negri, G. Rozza, A. Manzoni and A. Quarteroni. Reduced Basis Method for Parametrized Elliptic Optimal Control Problems. SIAM Journal on Scientific Computing, 35(5): A2316-A2340, 2013.
```

In [ ]:
import gmsh
from mpi4py import MPI
from dolfinx.io import XDMFFile
from multiphenicsx.mesh.utils import gmsh_to_fenicsx

In [ ]:
assert MPI.COMM_WORLD.size == 1, "This mesh generation notebook is supposed to be run in serial"

### Geometrical parameters and related quantities

In [ ]:
L1 = 1.0
L2 = 3.0
H = 1.0
lcar = 0.05

### Create mesh with gmsh

In [ ]:
gmsh.initialize()
gmsh.model.add("mesh")
p0 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, lcar)
p1 = gmsh.model.geo.addPoint(L1, 0.0, 0.0, lcar)
p2 = gmsh.model.geo.addPoint(L1 + L2, 0.0, 0.0, lcar)
p3 = gmsh.model.geo.addPoint(L1 + L2, H, 0.0, lcar)
p4 = gmsh.model.geo.addPoint(L1, H, 0.0, lcar)
p5 = gmsh.model.geo.addPoint(0.0, H, 0.0, lcar)
l0 = gmsh.model.geo.addLine(p0, p1)
l1 = gmsh.model.geo.addLine(p1, p4)
l2 = gmsh.model.geo.addLine(p4, p5)
l3 = gmsh.model.geo.addLine(p5, p0)
l4 = gmsh.model.geo.addLine(p1, p2)
l5 = gmsh.model.geo.addLine(p2, p3)
l6 = gmsh.model.geo.addLine(p3, p4)
line_loop_rectangle_left = gmsh.model.geo.addCurveLoop([l0, l1, l2, l3])
line_loop_rectangle_right = gmsh.model.geo.addCurveLoop([l4, l5, l6, -l1])
rectangle_left = gmsh.model.geo.addPlaneSurface([line_loop_rectangle_left])
rectangle_right = gmsh.model.geo.addPlaneSurface([line_loop_rectangle_right])
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [l0, l4, l5, l6, l2, l3], 1)
gmsh.model.addPhysicalGroup(2, [rectangle_left], 1)
gmsh.model.addPhysicalGroup(2, [rectangle_right], 2)
gmsh.model.mesh.generate(2)

### Convert to a dolfinx mesh

In [ ]:
mesh, subdomains, boundaries = gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

### Save mesh, subdomains and boundaries

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "rectangle.xdmf", "w") as file:
    file.write_mesh(mesh)
    file.write_meshtags(subdomains)
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    file.write_meshtags(boundaries)